In [1]:
from sas7bdat import SAS7BDAT
import pandas as pd
import pickle
import time
import datetime
import numpy as np
import statsmodels.formula.api as sm
import statsmodels.regression.linear_model as sm
import statsmodels.api as sm

from statsmodels.api import add_constant

In [ ]:
# 本篇為計算個股的beta 本資料為月資料 頻率為60期計算一次 滾動式計算
# msf為個股資料 factor為大盤資料


In [2]:
msf =  pd.read_pickle(r"C:\Users\user\OneDrive\桌面\美國資料\CRSP\msf.pkl")
factor = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\美國資料\CRSP\factors_monthly.pkl")

In [3]:
def report_day(a): 
    new_time = datetime.datetime(1960, 1, 1) + datetime.timedelta(days = a)
    return(new_time)

In [5]:
msf1.iloc[:,0:15]
bdata = msf1.drop(columns=['ISSUNO','HEXCD',
 'HSICCD','BID',
 'ASK',
 'SHROUT',
 'CFACPR',
 'CFACSHR',
 'ALTPRC',
 'SPREAD',
 'ALTPRCDT',"BIDLO","ASKHI","PRC","VOL",
 'RETX'],inplace=False)

In [7]:
cdata = bdata.reset_index(drop = True)
cdata.insert(5,"date",None)
cdata.insert(6,"RET_A",None)
cdata.insert(7,"mktrf",None)
cdata.insert(8,"rf",None)
cdata.insert(9,"beta_60",None)
cdata.insert(10,"beta_24",None)

In [8]:
r = cdata.columns.tolist().index("RET")
mr = cdata.columns.tolist().index("mktrf")
stpr = cdata.columns.tolist().index("RET_A")
rf = cdata.columns.tolist().index("rf")

In [9]:
pno_list = np.unique(cdata["PERMNO"].tolist()).tolist()

15189

In [10]:
cdata["date"] = pd.to_timedelta(cdata['DATE'], unit='d') + pd.datetime(1960, 1, 1)
cdata['year'] = cdata['date'].dt.year
cdata['month'] = cdata['date'].dt.month

<ipython-input-10-4fba98ae6b99>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  cdata["date"] = pd.to_timedelta(cdata['DATE'], unit='d') + pd.datetime(1960, 1, 1)


In [18]:
cdata

,CUSIP,PERMNO,PERMCO,DATE,RET,date,RET_A,mktrf,rf,beta_60,beta_24,year,month
0,36720410,10001.0,7953.0,16009.0,-0.130435,2003-10-31,None,None,None,None,None,2003,10
1,36720410,10001.0,7953.0,16037.0,-0.005000,2003-11-28,None,None,None,None,None,2003,11
2,36720410,10001.0,7953.0,16070.0,-0.003350,2003-12-31,None,None,None,None,None,2003,12
3,36720410,10001.0,7953.0,16100.0,0.010084,2004-01-30,None,None,None,None,None,2004,1
4,36720410,10001.0,7953.0,16128.0,0.079867,2004-02-27,None,None,None,None,None,2004,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380960,88160R10,93436.0,53453.0,21791.0,-0.066222,2019-08-30,None,None,None,None,None,2019,8
1380961,88160R10,93436.0,53453.0,21822.0,0.067639,2019-09-30,None,None,None,None,None,2019,9
1380962,88160R10,93436.0,53453.0,21853.0,0.307427,2019-10-31,None,None,None,None,None,2019,10
1380963,88160R10,93436.0,53453.0,21882.0,0.047695,2019-11-29,None,None,None,None,None,2019,11


In [13]:
fdata = factor.drop(factor.columns[7:9],axis = 1)
fdata = fdata.drop(factor.columns[2:4],axis = 1)
fdata = fdata.drop(factor.columns[0],axis = 1)

In [19]:
cfdata = pd.merge(cdata,fdata, on=['year','month']) 
cfdata

,CUSIP,PERMNO,PERMCO,DATE,RET,date,RET_A,mktrf_x,rf_x,beta_60,beta_24,year,month,mktrf_y,rf_y
0,36720410,10001.0,7953.0,16009.0,-0.130435,2003-10-31,None,None,None,None,None,2003,10,0.0608,0.0007
1,05978R10,10002.0,7954.0,16009.0,0.021854,2003-10-31,None,None,None,None,None,2003,10,0.0608,0.0007
2,23326910,10012.0,7969.0,16009.0,0.248062,2003-10-31,None,None,None,None,None,2003,10,0.0608,0.0007
3,00103110,10025.0,7975.0,16009.0,-0.152542,2003-10-31,None,None,None,None,None,2003,10,0.0608,0.0007
4,46603210,10026.0,7976.0,16009.0,-0.002515,2003-10-31,None,None,None,None,None,2003,10,0.0608,0.0007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380960,92835K10,93426.0,53443.0,21914.0,-0.011915,2019-12-31,None,None,None,None,None,2019,12,0.0277,0.0014
1380961,G3323L10,93427.0,53445.0,21914.0,0.071913,2019-12-31,None,None,None,None,None,2019,12,0.0277,0.0014
1380962,12503M10,93429.0,53447.0,21914.0,0.009251,2019-12-31,None,None,None,None,None,2019,12,0.0277,0.0014
1380963,78513510,93434.0,53427.0,21914.0,-0.070796,2019-12-31,None,None,None,None,None,2019,12,0.0277,0.0014


In [29]:
cfdata["RET_A"] = cfdata["RET"] - cfdata["rf_y"]
cfdata = cfdata.reindex(columns=['CUSIP','PERMNO','PERMCO','DATE','RET','date',
                                "RET_A","mktrf_y","rf_y","beta_60","beta_24","year","month"])
pn = cfdata["PERMNO"].tolist()
len(pno_list)
# 有15189間證券要計算

15189

In [30]:
cfdata

,CUSIP,PERMNO,PERMCO,DATE,RET,date,RET_A,mktrf_y,rf_y,beta_60,beta_24,year,month
0,36720410,10001.0,7953.0,16009.0,-0.130435,2003-10-31,-0.131135,0.0608,0.0007,None,None,2003,10
1,05978R10,10002.0,7954.0,16009.0,0.021854,2003-10-31,0.021154,0.0608,0.0007,None,None,2003,10
2,23326910,10012.0,7969.0,16009.0,0.248062,2003-10-31,0.247362,0.0608,0.0007,None,None,2003,10
3,00103110,10025.0,7975.0,16009.0,-0.152542,2003-10-31,-0.153242,0.0608,0.0007,None,None,2003,10
4,46603210,10026.0,7976.0,16009.0,-0.002515,2003-10-31,-0.003215,0.0608,0.0007,None,None,2003,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380960,92835K10,93426.0,53443.0,21914.0,-0.011915,2019-12-31,-0.013315,0.0277,0.0014,None,None,2019,12
1380961,G3323L10,93427.0,53445.0,21914.0,0.071913,2019-12-31,0.070513,0.0277,0.0014,None,None,2019,12
1380962,12503M10,93429.0,53447.0,21914.0,0.009251,2019-12-31,0.007851,0.0277,0.0014,None,None,2019,12
1380963,78513510,93434.0,53427.0,21914.0,-0.070796,2019-12-31,-0.072196,0.0277,0.0014,None,None,2019,12


In [38]:
# 總數為15189間 此處僅以前100間為例
for j in range(0,10):
#     print(j)
    pno = pno_list[j]
    t1 = cfdata[cfdata["PERMNO"]==pno]
    for i in range(0,len(t1)):
        t2 = t1.iloc[0:i,6:8]
        t3 = t2.dropna()
        t3 = t3.reset_index(drop = True)
        if len(t3) >= 60:
            t4 = t3.iloc[-60:,:]
            xdd = t4["mktrf_y"]
            yd = t4["RET_A"]
            model = sm.OLS(yd.astype(float),xdd.astype(float)).fit()
            b = model.params[0]
            ro = t1.iloc[i,:].name
            cfdata.iloc[ro,9] = b
            
        else:
            pass


        

end_time = time.time()

In [39]:
# beta_60即該證券的滾動式beta值
cfdata[cfdata["PERMNO"]==pno_list[3]].tail()

,CUSIP,PERMNO,PERMCO,DATE,RET,date,RET_A,mktrf_y,rf_y,beta_60,beta_24,year,month
1081978,00103110,10025.0,7975.0,20727.0,-0.011121,2016-09-30,-0.011321,0.0025,0.0002,1.10039,None,2016,9
1089281,00103110,10025.0,7975.0,20758.0,0.003932,2016-10-31,0.003732,-0.0202,0.0002,0.999825,None,2016,10
1096565,00103110,10025.0,7975.0,20788.0,0.075764,2016-11-30,0.075664,0.0486,0.0001,0.805716,None,2016,11
1103851,00103110,10025.0,7975.0,20818.0,-0.014849,2016-12-30,-0.015149,0.0181,0.0003,0.828589,None,2016,12
1111140,00103110,10025.0,7975.0,20850.0,NaN,2017-01-31,NaN,0.0194,0.0004,0.803504,None,2017,1
